# Deep learning for dynamic network analysis (DLDNA) - Project

Dolphins: R. ARNAUD M. DELPLANQUE A. KARILA-COHEN A. RAMPOLDI

Comprehensive soil classification dataset: https://www.kaggle.com/datasets/ai4a-lab/comprehensive-soil-classification-datasets/code

CNN puis GAN puis CyGAN

Binary classification : Binary CrossEntropy Loss $ \mathcal{L}_{\text{BCE}}(y,\hat y)
= - \left[ y \log(\hat y) + (1-y)\log(1-\hat y) \right]
 $ 

Rappel: le learning rate $ \alpha $ est le pas de mise à jour lors de la descente de gradient. Formule de la descente de gradient $ L(\theta_{n+1})= L(\theta_{n})-\alpha \nabla  L(\theta_{n}) $

In [ ]:
import warnings
warnings.filterwarnings('ignore')  # Suppress warnings to keep notebook clean

import random
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

# General
SEED = 42
BATCH_SIZE = 32
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TRAIN_RATIO  = 0.7
VAL_RATIO    = 0.1
TEST_RATIO   = 0.2

LEARNING_RATE = 0.01

# Use parameters for seed and device
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
device = torch.device(DEVICE)
print(f"Params loaded. Device: {DEVICE}")

Params loaded. Device: cpu


Penser à convertir les X et y en tenseur torch avant de procéder au datasplit via ``sklearn.model_selection.train_test_split`` <br>
```X = torch.FloatTensor(X) ``` et ```y = torch.FloatTensor(y) ``` <br

In [ ]:
# from sklearn.model_selection import train_test_split

# # First split: separate out test set (30% of total)
# X_train, X_temp, y_train, y_temp = train_test_split(
#     X, y, test_size=0.3, random_state=SEED  # 70% train, 30% temp
# )
# # Second split: split temp into validation (10%) and test (20%)
# X_val, X_test, y_val, y_test = train_test_split(
#     X_temp, y_temp, test_size=2/3, random_state=SEED  # 10% val, 20% test
# )
# print(f"Training samples: {len(X_train)}")  # Show split sizes
# print(f"Validation samples: {len(X_val)}")
# print(f"Test samples: {len(X_test)}")

**Notes**
Standardizer les data pour RGB et passer de 0 à 255 à 0 à 1

**Definition of the MLP**

In [ ]:
regression_model = MLPRegressor().to(device)  # Create model and move to device
regression_loss_fn = nn.BCELoss()  # Binary Cross Entropy loss
regression_optimizer = Adam(regression_model.parameters(), lr=0.01)  # Adam optimizer

**Grid search (hyperparameters optimisation for Adam GD): hidden size and learning rate**

If the learning rate is too big, the gradient descent cannot be stable. In a contrary if it is too small, the learning can be slow

In [13]:
learning_rates = [1e-1, 1e-2, 1e-3, 1e-4, 5e-4] 
hidden_sizes_options = [(32,16),(64,32), (128,64)]

def grid_search_hyperparameters(
    train_loader,
    val_loader,
    learning_rates,
    hidden_sizes_options,
    device,
    epochs=10,
    base_model=None,
    model_fn=None,
    save_path="best_model.pth",
):
    """
    Grid search over learning rates and hidden layer sizes.
    - model_fn: callable taking hidden_sizes (e.g., (h1, h2)) and returning an nn.Module (on CPU).
    - hidden_sizes_options: list of tuples like [(64,32), (128,64), ...]
    Saves the globally best model (by validation accuracy) to 'save_path'.
    Returns: (results_list, best_cfg_dict, best_model_loaded)
    """
    assert model_fn is not None, "Provide model_fn(hidden_sizes) -> nn.Module"

    results = []  # Store all results
    best_val_acc = -1.0  # Track best validation accuracy
    best_cfg = None  # Track best configuration
    best_state = None  # Track best model state

    for lr in learning_rates:  # Loop over learning rates
        for hidden_sizes in hidden_sizes_options:  # Loop over architectures
            print(f"Testing: lr={lr}, hidden_sizes={hidden_sizes}")

            model = model_fn(hidden_sizes).to(device)  # Create model
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # Create optimizer
            loss_fn = nn.CrossEntropyLoss()  # Define loss function

            # Train model
            _, _, _, train_accuracies, val_accuracies = train_with_validation(
                model=model,  # Model to train
                train_loader=train_loader,  # Training data
                val_loader=val_loader,  # Validation data
                optimizer=optimizer,  # Optimizer
                loss_fn=loss_fn,  # Loss function
                device=device,  # Device
                epochs=epochs,  # Number of epochs
                task_type='classification'  # Task type
            )

            cur_best_val = max(val_accuracies)  # Get best validation accuracy

            # Store results
            results.append({
                'lr': lr,  # Learning rate
                'hidden_sizes': hidden_sizes,  # Architecture
                'best_val_acc': cur_best_val,  # Best validation accuracy
                'final_train_acc': train_accuracies[-1],  # Final training accuracy
                'final_val_acc': val_accuracies[-1]  # Final validation accuracy
            })

            print(f"Best validation accuracy: {cur_best_val:.2f}%")

            # Update best model if this is better
            if cur_best_val > best_val_acc:
                best_val_acc = cur_best_val  # Update best accuracy
                best_cfg = {'lr': lr, 'hidden_sizes': hidden_sizes}  # Update best config
                best_state = {k: v.cpu() for k, v in model.state_dict().items()}  # Save state to CPU
                if save_path is not None:
                    torch.save(best_state, save_path)  # Save to disk
                    print(f"Saved new best model to: {save_path}")

            del model  # Free memory
            if torch.cuda.is_available():
                torch.cuda.empty_cache()  # Clear CUDA cache

    # Rebuild best model
    best_model = None
    if best_state is not None:
        best_model = model_fn(best_cfg['hidden_sizes']).to(device)  # Create model
        best_model.load_state_dict(best_state)  # Load best weights


    return results, best_cfg, best_model
